In [98]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [99]:
url ='https://www.bikedekho.com/new-bikes'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
r=requests.get(url,headers=headers).text
driver = webdriver.Firefox()
driver.get(url)


In [100]:
button = driver.find_elements(By.XPATH, "//span[contains(text(),'View All Brands')]")
button[0].click()

In [101]:
# soup = BeautifulSoup(driver,'lxml')
soup = BeautifulSoup(driver.page_source,"html.parser")


In [102]:
links_company = []
whitebox = soup.find_all('li', class_ = 'gsc_col-xs-4 gsc_col-sm-3 gsc_col-md-3 gsc_col-lg-2')
for i in range(44):
    A = whitebox[i].find_all('a',href = True)
    for i in A:
        links_company.append('https://www.bikedekho.com'+ i['href'])

In [103]:
len(links_company)

44

In [104]:
company_name = []
bike_name  = []
price = []
milege = []
links_bike = []
for i in links_company:
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    soup = BeautifulSoup(r,'lxml')
    len_bike = len(soup.find_all('div', class_ = 'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'))
    A = soup.find_all('h3')[0:len_bike]

    for i in range(len_bike):
        try:
            bike_name.append(soup.find_all('h3')[i].text)
        except:
            bike_name.append('Na')
        
        try:
            price.append(soup.find_all('div',class_ = 'price')[i].text)
        except:
            price.append('Na')
        try:
            if 'kmpl' in  soup.find_all('div',class_ = 'dotlist')[i].text :
                M =  soup.find_all('div',class_ = 'dotlist')[i].text
                milege.append((M.split(" ")[0]))
            else:
                milege.append('Na')
        except:
            milege.append('Na')


        try:
            company_name.append(soup.find_all('h1')[0].text)
        except:
            company_name.append("Na")
        
    
    for i in range(len_bike):
        for link in A[i].find_all('a',href = True):
            links_bike.append('https://www.bikedekho.com'+ link['href'])

In [105]:
Engine = []
power = []
Torque = [] 

for i in links_bike:
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    soup = BeautifulSoup(r,'lxml')
 
    try:
        for i in soup.find_all('td', class_ = 'gsc_col-xs-12 textHold')[0]:
            
            Engine.append(i.text)
    except:
        Engine.append('Na')

    try:
        # Engine =[power.append(i.text) for i in soup.find_all('td', class_ = 'gsc_col-xs-12 textHold') if "PS" in i.text]
        
        
        j = soup.find_all("td",{"class" : "gsc_col-xs-12 textHold"})

        if 'PS'  in j[1].text:
            power.append(j[1].text)
        elif 'PS'  in j[2].text:
            power.append(j[2].text)
        elif 'PS' in j[3].text:
            power.append(j[3].text)
        elif 'PS'  in j[4].text:
            power.append(j[4].text)
        elif 'PS' in j[5].text:
            power.append(j[5].text)
        else:
            power.append('NA')
            
            
    except:
        power.append('Na')
    
    try:
        # Engine =[power.append(i.text) for i in soup.find_all('td', class_ = 'gsc_col-xs-12 textHold') if "PS" in i.text]
        
        
        j = soup.find_all("td",{"class" : "gsc_col-xs-12 textHold"})

        if 'Nm'  in j[1].text:
            Torque.append(j[1].text)
        elif 'Nm'  in j[2].text:
            Torque.append(j[2].text)
        elif 'Nm'  in j[3].text:
            Torque.append(j[3].text)
        elif 'Nm' in j[4].text:
            Torque.append(j[4].text)
        elif 'Nm' in j[5].text:
            Torque.append(j[5].text)
        else:
            Torque.append('NA')
            
            
    except:
        Torque.append('Na')

In [106]:
col=['company_name','bike_name','price','milege','Engine','power','Torque'] 
df=pd.DataFrame({'company_name': company_name, 'bike_name': bike_name,'price':price,'milege':milege,'Engine':Engine,'power':power,'Torque':Torque})
df 


,company_name,bike_name,price,milege,Engine,power,Torque
0,TVS Bikes,TVS Apache RTR 160,Rs.1.18 - 1.25 Lakh*,Na,159.7 cc,16.04 PS,13.85 Nm
1,TVS Bikes,TVS Raider,"Rs.85,973 - 99,990*",67,124.8 cc,11.38 PS,11.2 Nm
2,TVS Bikes,TVS Apache RTR 160 4V,Rs.1.22 - 1.45 Lakh*,47.61,164.9 cc,19.2 PS,14.2 Nm
3,TVS Bikes,TVS Apache RR 310,Rs.2.65 Lakh*,33.1,312.2 cc,34 PS,27.3 Nm
4,TVS Bikes,TVS NTORQ 125,"Rs.79,956 - 99,961*",54.33,124.8 cc,9.38 PS,10.5 Nm
...,...,...,...,...,...,...,...
344,Moto Morini Bikes,Moto Morini X-Cape,Rs.7.20 - 7.50 Lakh*,Na,649 cc,60.8 PS,54 Nm
345,Zontes Bikes,Zontes 350R,Rs.3.15 - 3.25 Lakh*,Na,348 cc,38.52 PS,32.8 Nm
346,Zontes Bikes,Zontes GK350,Rs.3.37 - 3.47 Lakh*,31.25,348 cc,38.52 PS,32 Nm
347,Zontes Bikes,Zontes 350T,Rs.3.37 - 3.67 Lakh*,Na,348 cc,39.3 PS,32 Nm


In [107]:
df[['min_price', 'max_price']] = df.price.str.split("-", expand = True)
df = df.drop(columns = ['price']) 

In [108]:
df

,company_name,bike_name,milege,Engine,power,Torque,min_price,max_price
0,TVS Bikes,TVS Apache RTR 160,Na,159.7 cc,16.04 PS,13.85 Nm,Rs.1.18,1.25 Lakh*
1,TVS Bikes,TVS Raider,67,124.8 cc,11.38 PS,11.2 Nm,"Rs.85,973","99,990*"
2,TVS Bikes,TVS Apache RTR 160 4V,47.61,164.9 cc,19.2 PS,14.2 Nm,Rs.1.22,1.45 Lakh*
3,TVS Bikes,TVS Apache RR 310,33.1,312.2 cc,34 PS,27.3 Nm,Rs.2.65 Lakh*,None
4,TVS Bikes,TVS NTORQ 125,54.33,124.8 cc,9.38 PS,10.5 Nm,"Rs.79,956","99,961*"
...,...,...,...,...,...,...,...,...
344,Moto Morini Bikes,Moto Morini X-Cape,Na,649 cc,60.8 PS,54 Nm,Rs.7.20,7.50 Lakh*
345,Zontes Bikes,Zontes 350R,Na,348 cc,38.52 PS,32.8 Nm,Rs.3.15,3.25 Lakh*
346,Zontes Bikes,Zontes GK350,31.25,348 cc,38.52 PS,32 Nm,Rs.3.37,3.47 Lakh*
347,Zontes Bikes,Zontes 350T,Na,348 cc,39.3 PS,32 Nm,Rs.3.37,3.67 Lakh*


In [109]:

df=df[~df['power'].isin(['Na'])]
df=df[~df['power'].isin(['NA'])]

In [97]:
df.to_csv('bike_dekho.csv')